# 📘 word_full_pipeline_v8.3 — Word → CSV → YAML → RST(模块化) → Sphinx(HTML)

本版本在 v8.1 的基础上，**合并 v6.2 的完整抽取逻辑**，实现：
- ✅ 从 Word 自动抽取 **命令标题/描述**、**参数（含嵌套表 valmap）**、**示例（段落/表格）**；
- ✅ 生成 YAML，并进行基本校验；
- ✅ 生成模块化 RST（带 `_cmd_...` 锚点，支持分组 TOC）；
- ✅ 一键初始化 Sphinx 并构建 HTML（含 `docutils<0.21` 兼容、Windows 编码安全）。

> 使用前请将 `AT_Commands.docx` 放在与本 Notebook 同目录。
>
运行顺序：Step 0 → Step 6（或直接执行 `run_all(clean=True)`）。

## Step 0 — 安装依赖（首次运行需要）

In [18]:
!pip install -q python-docx pandas pyyaml jinja2 sphinx sphinx-book-theme sphinx-copybutton "docutils<0.21" lxml
print("✅ 依赖安装完成（含 docutils<0.21 兼容）")

✅ 依赖安装完成（含 docutils<0.21 兼容）


## Step 0.5 — 配置与通用工具（路径、日志、目录查看）

In [19]:
import os, re, json, traceback, datetime, subprocess, sys, shutil
import pandas as pd
from lxml import etree
from docx import Document
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl

# === 路径配置 ===
IN_WORD = "AT_Commands.docx"
DATA_DIR = "data"
CSV_OUT  = os.path.join(DATA_DIR, "at_extracted_commands.csv")
YAML_OUT = os.path.join(DATA_DIR, "at_all_commands.yaml")
RST_DIR  = os.path.join(DATA_DIR, "rst_output")

# 仿 ESP-AT：多语言目录（此处用 zh_CN）
DOCS_ROOT = "docs"
LANG = "zh_CN"
SRC_DIR = os.path.join(DOCS_ROOT, LANG, "source")
BUILD_HTML = os.path.join(DOCS_ROOT, LANG, "build", "html")
LOG_PATH = "parse_log.txt"

os.makedirs(DATA_DIR, exist_ok=True)
open(LOG_PATH, "w", encoding="utf-8").write("")

def log(msg: str):
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.datetime.now().isoformat(timespec='seconds')}] {msg}\n")

def print_tree(root_path):
    if not os.path.exists(root_path):
        print(f"(不存在) {root_path}")
        return
    for dirpath, dirnames, filenames in os.walk(root_path):
        level = dirpath.replace(root_path, "").count(os.sep)
        indent = "  " * level
        print(f"{indent}{os.path.basename(dirpath)}/")
        subindent = "  " * (level + 1)
        for f in filenames:
            print(f"{subindent}{f}")

print("✅ 配置就绪；输出目录：data/、docs/zh_CN/")

✅ 配置就绪；输出目录：data/、docs/zh_CN/


## Step 1 — Word → CSV（**完整抽取**：命令/描述 + 参数（嵌套表） + 示例）
核心：
- 顺序遍历段落与表格；
- 识别命令标题、参数标题（连续表）、示例标题；
- 参数值优先从第三列嵌套表提取，其次从文本模糊匹配；
- 示例支持“示例段落”和“示例表格”。

In [20]:
CMD_LINE_PAT = re.compile(r'^\s*(AT[\+\w\-]+(?:[?=][\w<>,\s\-\+\.\:\(\)]*)?)\s*(?::|：)?\s*(.*)$', re.I)
PARAM_HEADING_PAT = re.compile(r'^\s*参数(说明|表|信息)?\s*[:：]?\s*$', re.I)
EXAMPLE_HEADING_PAT = re.compile(r'^\s*(示例|Examples?)\s*[:：]?\s*$', re.I)

def is_cmd_heading(text): return bool(CMD_LINE_PAT.match(text or ""))
def is_param_heading(text): return bool(PARAM_HEADING_PAT.match(text or ""))
def is_example_heading(text): return bool(EXAMPLE_HEADING_PAT.match(text or ""))

def iter_ordered_blocks(doc):
    body = doc._element.body
    tbl_idx = 0
    for child in body.iterchildren():
        if isinstance(child, CT_P):
            text = "".join([t.text for t in child.xpath('.//w:t') if t.text]).strip()
            yield ("p", text)
        elif isinstance(child, CT_Tbl):
            table_obj = doc.tables[tbl_idx]
            tbl_idx += 1
            yield ("tbl", table_obj)

def cell_plain_text(cell):
    return "\n".join([p.text.strip() for p in cell.paragraphs if p.text and p.text.strip()]).strip()

def find_nested_tbls_in_cell(cell):
    xml_str = cell._tc.xml
    root = etree.fromstring(xml_str.encode("utf-8"))
    ns = {"w": "http://schemas.openxmlformats.org/wordprocessingml/2006/main"}
    return root.findall(".//w:tbl", ns), ns

def tbl_rows_as_text(tbl, ns):
    rows = []
    for r in tbl.findall(".//w:tr", ns):
        cells = r.findall(".//w:tc", ns)
        row = ["".join(tn.text for tn in c.iterfind(".//w:t", ns) if tn.text).strip() for c in cells]
        rows.append(row)
    return rows

def looks_like_header(row):
    hdr = " ".join(row[:3])
    return any(k in hdr for k in ["参数","名称","Name","描述","说明","含义","取值","值","value","meaning","备注","范围"])

def nested_table_to_valmap(rows):
    if not rows: return {}
    start = 1 if looks_like_header(rows[0]) else 0
    kv = {}
    for r in rows[start:]:
        if not r: continue
        key = (r[0] or "").strip()
        val = " | ".join([c for c in r[1:] if c and c.strip()]) if len(r) > 1 else ""
        if key: kv[key] = val
    return kv

def cell_valmap_from_nested_table(cell):
    tbls, ns = find_nested_tbls_in_cell(cell); mapping = {}
    for t in tbls:
        rows = tbl_rows_as_text(t, ns); mapping.update(nested_table_to_valmap(rows))
    return mapping

def parse_enum_map_fuzzy(text):
    if not text: return {}
    segs = re.split(r"[,\uFF0C;\uFF1B\n]+", text.strip())
    m = {}
    for s in segs:
        s = s.strip()
        if not s: continue
        if ":" in s or "：" in s:
            k, v = re.split(r"[:：]", s, 1); k, v = k.strip(), v.strip()
        else:
            m2 = re.match(r"^(\S+)\s*(?:->|→|=>|-|—|\s)\s*(.+)$", s)
            if m2: k, v = m2.group(1).strip(), m2.group(2).strip()
            else:
                m3 = re.match(r"^([A-Za-z0-9\+\-\.]+)\s+(.+)$", s)
                if m3: k, v = m3.group(1).strip(), m3.group(2).strip()
                else: continue
        if k: m[k] = v
    return m

def extract_examples_from_block(blocks, start_idx):
    examples = []
    i = start_idx
    pending_cmd = None
    while i < len(blocks):
        t, o = blocks[i]
        if t == "p" and (is_cmd_heading(o) or is_param_heading(o) or is_example_heading(o)):
            if is_example_heading(o):
                i += 1
                continue
            break
        if t == "p":
            line = (o or "").strip()
            if not line:
                i += 1; continue
            if line.startswith("AT+"):
                if pending_cmd:
                    examples.append({"cmd": pending_cmd, "resp": ""})
                pending_cmd = line
            elif pending_cmd and (line.startswith("+") or line in ("OK","ERROR") or line.startswith("ERROR")):
                examples.append({"cmd": pending_cmd, "resp": line}); pending_cmd = None
        elif t == "tbl":
            tbl = o
            for r in tbl.rows:
                cols = r.cells
                if not any(c.text.strip() for c in cols):
                    continue
                cmd = cell_plain_text(cols[0]) if len(cols) > 0 else ""
                resp = cell_plain_text(cols[1]) if len(cols) > 1 else ""
                if cmd:
                    examples.append({"cmd": cmd, "resp": resp})
        i += 1
    if pending_cmd:
        examples.append({"cmd": pending_cmd, "resp": ""})
    return examples, i

def extract_word_to_csv(docx_path, csv_out):
    if not os.path.exists(docx_path):
        raise FileNotFoundError(f"未找到 Word 文件: {docx_path}")
    doc = Document(docx_path)
    seq = list(iter_ordered_blocks(doc))
    rows = []
    i = 0; order = 0
    while i < len(seq):
        t, o = seq[i]
        if t == "p" and is_cmd_heading(o):
            m = CMD_LINE_PAT.match(o)
            command = m.group(1).strip()
            title = (m.group(2) or "").strip()
            order += 1

            # 描述汇总
            desc_lines = []
            j = i + 1
            while j < len(seq):
                t2, o2 = seq[j]
                if t2 == "p" and (is_cmd_heading(o2) or is_param_heading(o2) or is_example_heading(o2)):
                    break
                if t2 == "p" and o2:
                    desc_lines.append(o2)
                elif t2 == "tbl":
                    break
                j += 1
            description = "\n".join(desc_lines).strip()

            # 参数解析
            params = []; table_count = 0
            k = j
            while k < len(seq):
                t3, o3 = seq[k]
                if t3 == "p" and is_cmd_heading(o3):
                    break
                if t3 == "p" and is_param_heading(o3):
                    k += 1
                    while k < len(seq) and seq[k][0] == "tbl":
                        table = seq[k][1]; table_count += 1
                        for r in table.rows:
                            cols = r.cells
                            if not any(c.text.strip() for c in cols):
                                continue
                            name = cell_plain_text(cols[0]) if len(cols) > 0 else ""
                            desc = cell_plain_text(cols[1]) if len(cols) > 1 else ""
                            valmap = {}
                            if len(cols) > 2:
                                valmap = cell_valmap_from_nested_table(cols[2]) or parse_enum_map_fuzzy(cell_plain_text(cols[2]))
                            if not valmap and len(cols) > 1:
                                valmap = cell_valmap_from_nested_table(cols[1]) or parse_enum_map_fuzzy(desc)
                            # 跳过表头
                            if name in ("参数","参数名","Name") and any(x in desc for x in ["描述","说明","Description","Meaning"]):
                                continue
                            if name or desc or valmap:
                                params.append({"name": name, "desc": desc, "valmap": valmap})
                        k += 1
                    continue
                k += 1

            # 示例解析
            examples = []
            k2 = j
            while k2 < len(seq):
                t4, o4 = seq[k2]
                if t4 == "p" and is_example_heading(o4):
                    ex, stop_idx = extract_examples_from_block(seq, k2+1)
                    examples.extend(ex); k2 = stop_idx
                elif t4 == "p" and is_cmd_heading(o4):
                    break
                else:
                    k2 += 1

            # 压回 CSV 结构
            ex_cmd_join = "|".join([e["cmd"] for e in examples]) if examples else command
            ex_resp_join = "|".join([e["resp"] for e in examples]) if examples else ""

            rows.append({
                "命令": command,
                "命令标题": title,
                "命令类型": "执行;查询;测试",
                "命令格式": command,
                "示例命令": ex_cmd_join,
                "示例响应": ex_resp_join,
                "功能描述": description or title,
                "备注": "",
                "参数JSON": json.dumps(params, ensure_ascii=False),
                "顺序": order,
                "表数量": table_count
            })
            i = max(j, k, k2); continue
        i += 1

    df = pd.DataFrame(rows)
    df.to_csv(CSV_OUT, index=False, encoding="utf-8-sig")
    print(f"✅ 提取 {len(df)} 条命令 → {CSV_OUT}")
    print(f"📝 解析日志：{LOG_PATH}")
    df.head(3)

_ = extract_word_to_csv(IN_WORD, CSV_OUT)
pd.read_csv(CSV_OUT).head(2)

C:\Users\txiab\AppData\Local\Temp\ipykernel_48908\2016199789.py:67: DeprecationWarning: 'maxsplit' is passed as positional argument
  k, v = re.split(r"[:：]", s, 1); k, v = k.strip(), v.strip()


✅ 提取 44 条命令 → data\at_extracted_commands.csv
📝 解析日志：parse_log.txt


,命令,命令标题,命令类型,命令格式,示例命令,示例响应,功能描述,备注,参数JSON,顺序,表数量
0,ATI,获取模组厂商信息,执行;查询;测试,ATI,ATI\nNEOWAY\nN706\nV001\nOK,厂家信息\n模组型号\n版本号,获取模组厂商信息，包括厂家、型号和版本。\n命令格式,NaN,"[{""name"": ""<manufacturer>"", ""desc"": ""模组厂商信息、产品...",1,1
1,AT+GMR,查询版本信息,执行;查询;测试,AT+GMR,AT+GMR\n+GMR: N706-R004-STD-BZ-003\nOK,查询软件版本,查询软件版本信息。\n命令格式,NaN,"[{""name"": ""<reversion>"", ""desc"": ""模组软件版本信息"", ""...",2,1


## Step 2 — CSV → YAML（合并示例 & 参数 JSON）+ 校验（2.5）

In [21]:
import yaml

def csv_to_yaml(csv_path, yaml_path):
    df = pd.read_csv(csv_path, dtype=str).fillna("")
    objs = []
    for _, r in df.iterrows():
        params = json.loads(r.get("参数JSON","[]")) if r.get("参数JSON") else []
        # 处理示例（'|' 串 → 列表[{cmd, resp}]）
        cparts = [x.strip() for x in r.get("示例命令","" ).split("|") if x.strip()]
        rparts = [x.strip() for x in r.get("示例响应","" ).split("|")]
        examples = []
        for idx, c in enumerate(cparts):
            resp = rparts[idx] if idx < len(rparts) else ""
            examples.append({"cmd": c, "resp": resp})

        objs.append({
            "command": r["命令"],
            "title": r["命令标题"],
            "type": [t.strip() for t in r["命令类型"].split(";") if t.strip()],
            "formats": [f.strip() for f in r["命令格式"].split("|") if f.strip()] or [r["命令格式"]],
            "parameters": params,
            "examples": examples,
            "description": r.get("功能描述",""),
            "notes": r.get("备注",""),
            "meta": {"order": int(r.get("顺序","0") or 0), "tables": int(r.get("表数量","0") or 0)}
        })
    objs.sort(key=lambda x: x["meta"]["order"])  # 保序
    with open(yaml_path, "w", encoding="utf-8") as f:
        yaml.safe_dump({"commands": objs}, f, allow_unicode=True, sort_keys=False)
    print(f"✅ 已生成 YAML → {yaml_path}")
    return yaml_path

csv_to_yaml(CSV_OUT, YAML_OUT)

def validate_yaml(yaml_path):
    with open(yaml_path, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f) or {}
    cmds = data.get("commands", [])
    seen = set(); err = []
    for c in cmds:
        cmd = c.get("command","")
        if not cmd: err.append("缺少 command"); continue
        if cmd in seen: err.append(f"重复 command: {cmd}")
        seen.add(cmd)
        for p in c.get("parameters", []):
            if not p.get("name") and not p.get("desc") and not p.get("valmap"):  # 全空视为噪声
                err.append(f"{cmd}: 存在空参数行")
    if err:
        print("❌ YAML 校验失败：\n - " + "\n - ".join(err))
        raise SystemExit(1)
    print(f"✅ YAML 校验通过，共 {len(cmds)} 条命令")

validate_yaml(YAML_OUT)

✅ 已生成 YAML → data\at_all_commands.yaml
✅ YAML 校验通过，共 44 条命令


## Step 3 — YAML → 模块化 RST（带锚点 + 参数嵌套 valmap + 分组 TOC）

In [ ]:
from jinja2 import Template
from collections import defaultdict

def safe_stub(name: str) -> str:
    return name.replace("+","plus").replace("?","q").replace("=","eq").replace("/","sl")

PAGE_TMPL = Template('''
.. _cmd_{{ safe(cmd.command) }}:

{{ cmd.command }}
{{ '=' * cmd.command|length }}

**功能说明**
{{ cmd.description or cmd.title or '' }}

**命令语法**
::
{% for f in cmd.formats %}
   {{ f }}
{% endfor %}

**参数说明**

.. list-table::
   :header-rows: 1
   :widths: 18 32 50
   :class: cmd-param-table

   * - 参数名
     - 描述
     - 取值
{% for p in cmd.parameters %}
   * - {{ p.name or '—' }}
     - {{ p.desc or '—' }}
     - {% if p.valmap %}
       .. list-table::
          :header-rows: 1
          :widths: 25 75

          * - 值
            - 含义
{% for k, v in p.valmap.items() %}
          * - {{ k }}
            - {{ v }}
{% endfor %}
       {% else %} N/A {% endif %}
{% endfor %}

**示例**
.. code-block:: bash

{% for ex in cmd.examples %}
   {{ ex.cmd }}
{% if ex.resp %}
   {{ ex.resp }}
{% endif %}
{% if loop.nextitem %}
   # —— 以下示例 ——
{% endif %}
{% endfor %}
''')

def group_key(cmd_str):
    m = re.match(r'^AT\+([A-Z]+)', (cmd_str or "").upper())
    if not m: return "AT-OTHER"
    token = m.group(1)
    return f"AT-{token[:2]}" if len(token) >= 2 else "AT-OTHER"

def yaml_to_rst_modular(yaml_path, rst_root):
    if os.path.exists(rst_root):
        shutil.rmtree(rst_root)
    os.makedirs(rst_root, exist_ok=True)
    with open(yaml_path, 'r', encoding='utf-8') as f:
        data = yaml.safe_load(f)
    cmds = data.get('commands', [])

    groups = defaultdict(list)
    for c in cmds:
        rst = PAGE_TMPL.render(cmd=c, safe=safe_stub)
        fname = f"{safe_stub(c['command'])}.rst"
        with open(os.path.join(rst_root, fname), 'w', encoding='utf-8') as fo:
            fo.write(rst)
        groups[group_key(c['command'])].append(fname[:-4])

    # 分组索引 + 总索引
    for g, items in groups.items():
        grp_dir = os.path.join(rst_root, g)
        os.makedirs(grp_dir, exist_ok=True)
        glines = [g, '=' * len(g), '', ".. toctree::", "   :maxdepth: 1", '']
        for it in items:
            glines.append(f"   ../{it}")
        with open(os.path.join(grp_dir, 'index.rst'), 'w', encoding='utf-8') as fo:
            fo.write('\n'.join(glines))

    idx = ["AT 命令手册", "==========", '', ".. toctree::", "   :maxdepth: 2", '']
    for g in sorted(groups.keys()):
        idx.append(f"   {g}/index")
    with open(os.path.join(rst_root, 'index.rst'), 'w', encoding='utf-8') as fo:
        fo.write('\n'.join(idx))
    print(f"✅ 模块化 RST 已生成 → {rst_root}")

yaml_to_rst_modular(YAML_OUT, RST_DIR)
sorted(os.listdir(RST_DIR))[:10]

✅ 模块化 RST 已生成 → data\rst_output


['AT-CC',
 'AT-CE',
 'AT-CF',
 'AT-CG',
 'AT-CI',
 'AT-CL',
 'AT-CM',
 'AT-CN',
 'AT-CO',
 'AT-CP']

## Step 4 — 初始化 Sphinx（多语言结构 + 完整 conf.py + CSS）
自动创建 `docs/zh_CN/source/` 结构，复制 RST，并写入主题/扩展/CSS 配置。

In [23]:
# 清理旧文档
if os.path.exists(DOCS_ROOT):
    print("⚠️ 检测到旧 docs/，正在清理…")
    shutil.rmtree(DOCS_ROOT)
    print("✅ 已删除旧 docs/")

os.makedirs(SRC_DIR, exist_ok=True)
os.makedirs(os.path.join(SRC_DIR, '_static'), exist_ok=True)
os.makedirs(os.path.join(DOCS_ROOT, LANG, 'build', 'html'), exist_ok=True)

# 复制 RST
shutil.copytree(RST_DIR, SRC_DIR, dirs_exist_ok=True)

# 写 conf.py（完整）
conf_text = '''
project = 'AT Command Manual'
master_doc = 'index'
language = 'zh_CN'
extensions = [
    'sphinx.ext.autosectionlabel',
    'sphinx_copybutton',
]
autosectionlabel_prefix_document = True
html_theme = 'sphinx_book_theme'
html_static_path = ['_static']
def setup(app):
    app.add_css_file('custom.css')
'''
with open(os.path.join(SRC_DIR, 'conf.py'), 'w', encoding='utf-8') as f:
    f.write(conf_text)

# 写 custom.css
custom_css = '''
/* 提示块与表格样式 */
.cmd-param-table th { background: #f2f2f2; }
.cmd-param-table td, .cmd-param-table th { padding: 6px 10px; }
pre { background: #f8f9fa; border-radius: 6px; padding: 8px; }
'''
with open(os.path.join(SRC_DIR, '_static', 'custom.css'), 'w', encoding='utf-8') as f:
    f.write(custom_css)

print("✅ Sphinx 初始化完成并复制 RST + 注入主题与 CSS")
print_tree(DOCS_ROOT)

⚠️ 检测到旧 docs/，正在清理…
✅ 已删除旧 docs/
✅ Sphinx 初始化完成并复制 RST + 注入主题与 CSS
docs/
  zh_CN/
    build/
      html/
    source/
      ATD.rst
      ATE1.rst
      ATI.rst
      ATplusCCID.rst
      ATplusCCLK.rst
      ATplusCEREG.rst
      ATplusCESQ.rst
      ATplusCFUN.rst
      ATplusCGATT.rst
      ATplusCGDCONT.rst
      ATplusCGMM.rst
      ATplusCGSN.rst
      ATplusCIMI.rst
      ATplusCLCK.rst
      ATplusCMGD.rst
      ATplusCMGF.rst
      ATplusCMGL.rst
      ATplusCMGR.rst
      ATplusCMGS.rst
      ATplusCMGW.rst
      ATplusCMSS.rst
      ATplusCMUX.rst
      ATplusCNMI.rst
      ATplusCOPS.rst
      ATplusCPIN.rst
      ATplusCPMS.rst
      ATplusCPWD.rst
      ATplusCREG.rst
      ATplusCSCA.rst
      ATplusCSCS.rst
      ATplusCSDH.rst
      ATplusCSMP.rst
      ATplusCSMS.rst
      ATplusCSQ.rst
      ATplusGMM.rst
      ATplusGMR.rst
      ATplusGSN.rst
      ATplusIPR.rst
      ATplusNSTGETRSSI.rst
      ATplusNWDNS.rst
      ATplusNWENPWRSAVE.rst
      ATplusNWPWROFF.rst
   

## Step 5 — 构建 HTML（UTF-8 日志 & 失败自动回退 docutils 已在 Step 0 处理）
Windows 可能出现控制台编码问题，这里用 `encoding='utf-8', errors='ignore'` 避免乱码中断。

In [24]:
def build_html_fixed():
    print("📦 开始构建 HTML …")
    # 保证 index.rst 存在
    if not os.path.exists(os.path.join(SRC_DIR, 'index.rst')):
        with open(os.path.join(SRC_DIR, 'index.rst'), 'w', encoding='utf-8') as f:
            f.write("AT Manual\n=========\n\n.. toctree::\n   :maxdepth: 1\n\n")
            for fn in os.listdir(SRC_DIR):
                if fn.endswith('.rst') and fn != 'index.rst':
                    f.write(f"   {fn}\n")

    os.makedirs(BUILD_HTML, exist_ok=True)
    cmd = [sys.executable, '-m', 'sphinx', '-b', 'html', SRC_DIR, BUILD_HTML]
    p = subprocess.run(cmd, capture_output=True, text=True, encoding='utf-8', errors='ignore')
    print(p.stdout or p.stderr)
    index_path = os.path.join(BUILD_HTML, 'index.html')
    if os.path.exists(index_path):
        print(f"✅ 构建成功：{index_path}")
        return True
    print("❌ 构建失败，请查看日志。")
    return False

build_html_fixed()

📦 开始构建 HTML …
Running Sphinx v8.2.3
loading translations [zh_CN]... done
building [mo]: targets for 0 po files that are out of date
writing output... 
building [html]: targets for 64 source files that are out of date
updating environment: [new config] 64 added, 0 changed, 0 removed
reading sources... [  2%] AT-CC/index
reading sources... [  3%] AT-CE/index
reading sources... [  5%] AT-CF/index
reading sources... [  6%] AT-CG/index
reading sources... [  8%] AT-CI/index
reading sources... [  9%] AT-CL/index
reading sources... [ 11%] AT-CM/index
reading sources... [ 12%] AT-CN/index
reading sources... [ 14%] AT-CO/index
reading sources... [ 16%] AT-CP/index
reading sources... [ 17%] AT-CR/index
reading sources... [ 19%] AT-CS/index
reading sources... [ 20%] AT-GM/index
reading sources... [ 22%] AT-GS/index
reading sources... [ 23%] AT-IP/index
reading sources... [ 25%] AT-NS/index
reading sources... [ 27%] AT-NW/index
reading sources... [ 28%] AT-OTHER/index
reading sources... [ 30%] AT-X

True

## 🟢 Step 6 — 一键执行 `run_all(clean=True)`（全流程自动）

In [25]:
def run_all(clean=True):
    if clean and os.path.exists(DOCS_ROOT):
        print("⚠️ run_all: 清理旧 docs/ …")
        shutil.rmtree(DOCS_ROOT)
    _ = extract_word_to_csv(IN_WORD, CSV_OUT)
    csv_to_yaml(CSV_OUT, YAML_OUT)
    validate_yaml(YAML_OUT)
    yaml_to_rst_modular(YAML_OUT, RST_DIR)
    # 初始化 Sphinx
    os.makedirs(SRC_DIR, exist_ok=True)
    os.makedirs(os.path.join(SRC_DIR, '_static'), exist_ok=True)
    os.makedirs(os.path.join(DOCS_ROOT, LANG, 'build', 'html'), exist_ok=True)
    shutil.copytree(RST_DIR, SRC_DIR, dirs_exist_ok=True)
    # 写配置
    with open(os.path.join(SRC_DIR, 'conf.py'), 'w', encoding='utf-8') as f:
        f.write("""project='AT Command Manual'\nmaster_doc='index'\nlanguage='zh_CN'\nextensions=['sphinx.ext.autosectionlabel','sphinx_copybutton']\nautosectionlabel_prefix_document=True\nhtml_theme='sphinx_book_theme'\nhtml_static_path=['_static']\n\ndef setup(app):\n    app.add_css_file('custom.css')\n""")
    with open(os.path.join(SRC_DIR, '_static', 'custom.css'), 'w', encoding='utf-8') as f:
        f.write(".cmd-param-table th{background:#f2f2f2;} .cmd-param-table td,.cmd-param-table th{padding:6px 10px;} pre{background:#f8f9fa;border-radius:6px;padding:8px;}")
    ok = build_html_fixed()
    if ok:
        print("\n✅ 全流程完成。HTML 查看：", os.path.join(BUILD_HTML, 'index.html'))
        print("📝 解析日志：", LOG_PATH)
    else:
        print("❌ 构建未成功，请先单步检查 Step 1~3 的输出。")

print("准备就绪。按顺序运行 Step 0→6，或直接 run_all(clean=True)。")

准备就绪。按顺序运行 Step 0→6，或直接 run_all(clean=True)。
